In [1]:
import keras
from keras import layers
from keras import backend as k
from keras.models import Model
import numpy as np
k.clear_session()

c:\users\brad\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_shape = (28,28,1)
batch_size = 16
latent_dim = 2 #latent space shape is a 2-d plane corresponding to 2 node hidden layer values

In [3]:
input_img = keras.Input(shape = img_shape)
x = layers.Conv2D(32,3, padding = 'same', activation = 'relu')(input_img)
x = layers.Conv2D(64,3, padding = 'same', activation = 'relu', strides = (2,2))(x)
x = layers.Conv2D(64,3, padding = 'same', activation = 'relu')(x)
x = layers.Conv2D(64,3, padding = 'same', activation = 'relu')(x)
shape_before_flattening = k.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation = 'relu')(x)
print(x)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

Tensor("dense_1/Relu:0", shape=(?, 32), dtype=float32)


In [4]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = k.random_normal(shape = (k.shape(z_mean)[0], latent_dim), mean = 0., stddev = 1.)
    return z_mean + k.exp(z_log_var) * epsilon

In [5]:
z = layers.Lambda(sampling)([z_mean, z_log_var])
print(z)

Tensor("lambda_1/add:0", shape=(?, 2), dtype=float32)


In [6]:
decoder_input = layers.Input(k.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]), activation = 'relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32,3,padding = 'same', activation = 'relu', strides = (2,2))(x)
x = layers.Conv2D(1, 3 ,padding = 'same', activation = 'sigmoid')(x)
print(x)

Tensor("conv2d_5/Sigmoid:0", shape=(?, ?, ?, 1), dtype=float32)


In [7]:
decoder = Model(decoder_input, x)
z_decoded = decoder(z)
print(z_decoded)

Tensor("model_1/conv2d_5/Sigmoid:0", shape=(?, ?, ?, 1), dtype=float32)


In [8]:
class CustomVariationalLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = k.flatten(x)
        z_decoded = k.flatten(z_decoded)
        print(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * k.mean(1 + z_log_var - k.square(z_mean) - k.exp(z_log_var), axis = -1)
        return(k.mean(xent_loss + kl_loss)) #loss is average of classification error and the kullback liebler divergence
    
    def call(self, inputs):  # custom layer call override
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs = inputs)
        return x        

In [9]:
y = CustomVariationalLayer()([input_img, z_decoded])

Tensor("custom_variational_layer_1/Reshape_1:0", shape=(?,), dtype=float32)


In [10]:
from keras.datasets import mnist

In [11]:
vae = Model(input_img, y)
vae. compile(optimizer = 'rmsprop', loss = None)
vae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 28, 28, 32)   320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 14, 14, 64)   18496       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 14, 14, 64)   36928       conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_4 (

c:\users\brad\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Output "custom_variational_layer_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "custom_variational_layer_1" during training.
  


In [12]:
(x_train, _), (x_test, y_test) = mnist.load_data()

In [13]:
x_train = x_train.astype('float32')/255.
x_train = x_train.reshape(x_train.shape + (1,))

x_test = x_test.astype('float32')/255.
x_test = x_test.reshape(x_test.shape + (1,))

In [ ]:
vae.fit(x = x_train, y = None, shuffle = True, epochs = 10, batch_size = batch_size, validation_data = (x_test, None))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 27s 443us/step - loss: 0.2131 - val_loss: 0.1940
Epoch 2/10
60000/60000 [==============================] - 25s 411us/step - loss: 0.1916 - val_loss: 0.1890
Epoch 3/10
60000/60000 [==============================] - 24s 399us/step - loss: 0.1878 - val_loss: 0.1860
Epoch 4/10
60000/60000 [==============================] - 24s 396us/step - loss: 0.1854 - val_loss: 0.1846
Epoch 5/10
60000/60000 [==============================] - 24s 394us/step - loss: 0.1835 - val_loss: 0.1833
Epoch 6/10
60000/60000 [==============================] - 24s 403us/step - loss: 0.1823 - val_loss: 0.1821
Epoch 7/10
33168/60000 [===============>..............] - ETA: 9s - loss: 0.1813 